In [8]:
import datetime
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import csv
from pathlib import Path
from selenium.webdriver.chrome.service import Service as EdgeService

In [9]:
filename = "articles_info.csv" # имя файла, в который будем сохранять результат
executable_path = './msedgedriver.exe' # укажите ваш путь к chromedriver, который вы загрузили ранее
base_dir= "./data/" # укажите директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 ..." # ваш user-agent, узнать его можно тут: https://юзерагент.рф, смотреть через браузер Chrome
start_time = time() # время начала выполнения программы

In [10]:
def get_load_time(article_url, user_agent):
    #будем ждать 3 секунды, иначе выводить exception и присваивать константное значение
    try:
        # меняем значение заголовка. По умолчанию указано, что это python-код
        headers = {
            "User-Agent": user_agent
        }
        # делаем запрос по url статьи article_url
        response = requests.get(
            article_url, headers=headers, stream=True, timeout=3.000
        )
        # получаем время загрузки страницы
        load_time = response.elapsed.total_seconds()
    except Exception as e:
        print(e)
        load_time = ">3"
    return load_time

In [11]:
def write_to_file(output_list, filename, base_dir):
    for row in output_list:
        with open(Path(base_dir).joinpath(filename), "a") as csvfile:
            fieldnames = ["id", "load_time", "rank", "points", "title", "url", "comment"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(row)

In [13]:
def connect_to_base(browser, page_number):
    base_url = "https://news.ycombinator.com/news?p={}".format(page_number)
    for connection_attempts in range(1,4): # совершаем 3 попытки подключения
        try:
            browser.get(base_url)
            # ожидаем пока элемент table с id = 'hnmain' будет загружен на страницу
            # затем функция вернет True иначе False 
            WebDriverWait(browser, 5).until(
                EC.presence_of_element_located((By.ID, "hnmain"))
            )
            return True
        except Exception as e:
            print(e)
            print("Error connecting to {}.".format(base_url))
            print("Attempt #{}.".format(connection_attempts))
    return False

In [14]:
def parse_html(html, user_agent):
    soup = BeautifulSoup(html, "html.parser")
    output_list = []
   
    # ищем в объекте soup object id, rank, score и title статьи
    tr_blocks = soup.find_all("tr", class_="athing")
    article = 0
    for tr in tr_blocks:
        article_id = tr.get("id") # id
        article_url = tr.find_all("a")[1]["href"]

        # иногда статья располагается не на внешнем сайте, а на ycombinator
        # тогда article_url у нее не полный, а добавочный, с параметрами.
        # например item?id=200933. Для этих случаев будем добавлять url до полного
        if "item?id=" in article_url or "from?site=" in article_url:
            article_url = f"https://news.ycombinator.com/{article_url}"
        load_time = get_load_time(article_url, user_agent)
        # иногда рейтинга может не быть, поэтому воспользуемся try

        try:
            score = soup.find(id=f"score_{article_id}").string
        except Exception as e:
            print(e)
            score = "0 points"
            
        try:
            com = soup.find_all('a', href=f"item?id={article_id}")
            com = com[1].string
        except Exception as e:
            print(e)
            com = "0 comments"
           
        article_info = {
            "id": article_id,
            "load_time": load_time,
            "rank": tr.span.string,
            "points": score,
            "title": tr.find(class_="titleline").string,
            "url": article_url,
            "comment": com
        }

        # добавляем информацию о статье в список
        output_list.append(article_info)
        article += 1
    return output_list

In [123]:
#start_time = time() # время начала выполнения программы

# инициализируем веб драйвер
#browser = webdriver.Edge(
#    service=EdgeService(executable_path=executable_path)
#)

# перебираем страницы и собираем нужную информацию
#for page_number in range(10):
#    print("getting page " + str(page_number) + "...")
#    if connect_to_base(browser, page_number):
#        sleep(5)
#        output_list = parse_html(browser.page_source, user_agent)
#        write_to_file(output_list, filename, base_dir)
#
#    else:
#        print("Error connecting to hacker news")

# завершаем работу драйвера
#browser.close()
#sleep(1)
#browser.quit()
#end_time = time()
#elapsed_time = end_time - start_time
#print("run time: {} seconds".format(elapsed_time))

In [17]:
from concurrent.futures import ThreadPoolExecutor, wait

# Обернём процедуру парсинга страницы в функцию
def run_process(page_number, filename):
    browser = webdriver.Edge(
        service=EdgeService(executable_path=executable_path)
    )
    if connect_to_base(browser, page_number):
        sleep(5)
        output_list = parse_html(browser.page_source, user_agent)
        #write_to_file(output_list, filename, base_dir)
        save_to_db(output_list)
        browser.quit()
    else:
        print("Error connecting to hacker news")
        browser.quit()
       
# Глобальные переменные        
filename = "articles_info_new.csv" # имя файла, в который будем сохранять результат
executable_path = './msedgedriver.exe' # укажите ваш путь к chromedriver, который вы загрузили ранее
base_dir= "./data/" # укажите директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"

# Засечём время выполнения кода
start_time = time()

futures = []

# Запустим процесс парсинга на нескольких потоках одновременно
with ThreadPoolExecutor() as executor:
    for number in range(10):
        futures.append(
            executor.submit(run_process, number, filename)
        )
       
wait(futures)
end_time = time()
elapsed_time = end_time - start_time
print("Elapsed run time: {} seconds".format(elapsed_time))

'NoneType' object has no attribute 'string'
list index out of range
HTTPSConnectionPool(host='www.euronews.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.linkedin.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.lemonde.fr', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='avi.im', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='newsroom.lexmark.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='talktomehuman.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='www.euronews.com', port=443): Read timed out. (read timeout=3.0)
HTTPSConnectionPool(host='berthub.eu', port=443): Max retries exceeded with url: /articles/posts/on-long-term-software-development/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object

In [127]:
articles_data_new = pd.read_csv(
    './data/articles_info_new.csv',
    names=["id", "load_time", "rank", "points", "title", "url", "comment"],
    encoding='cp1252'
)

articles_data_new.shape

(300, 7)

In [128]:
articles_data_new

,id,load_time,rank,points,title,url,comment
0,42443456,0.674293,151.0,27 points,NaN,https://www.natesilver.net/p/save-daylight-sav...,54 comments
1,42457383,0.26473,152.0,41 points,NaN,https://pmc.ncbi.nlm.nih.gov/articles/PMC3536509/,17 comments
2,42487746,0.932203,153.0,53 points,NaN,https://www.theatlantic.com/ideas/archive/2024...,86 comments
3,42454359,0.535328,154.0,679 points,NaN,https://github.com/pwr-Solaar/Solaar,259 comments
4,42437201,0.862806,155.0,170 points,NaN,https://arstechnica.com/health/2024/12/huge-ma...,111 comments
...,...,...,...,...,...,...,...
295,42461539,0.548806,56.0,24 points,NaN,https://netboxlabs.com/blog/netbox-discovery-a...,discuss
296,42456802,0.27983,57.0,80 points,NaN,https://genesis-embodied-ai.github.io/,2 comments
297,42479661,0.088637,58.0,21 points,NaN,https://www.chicagotribune.com/2024/12/21/chic...,2 comments
298,42492949,0.670603,59.0,22 points,NaN,https://finance.yahoo.com/news/france-links-fi...,2 comments


In [4]:
import psycopg2
connection = psycopg2.connect(user='postgres',
                                  password=123580,
                                  host='localhost',
                                  port=5432,
                                  database="test_base")

In [5]:
connection.autocommit = True


# запись в бд происходит с помощью “курсора”. Мы пишем запрос и выполняем его с помощью execute
with connection.cursor() as cursor:
    cursor.execute("""
    DROP TABLE IF EXISTS articles;
    CREATE TABLE articles(
    id integer,
    load_time text,
    rank text,
    points text,
    title text,
    url text
)
""")

In [15]:
def save_to_db(output_list):
    with connection.cursor() as cursor:
		#пробегаемся по строкам с результатом и вставляем значения в строку. Для вставки данных в таблицу используем конструкцию INSERT INTO TABLE
        for row in output_list:
            cursor.execute("""
                INSERT INTO articles VALUES (
                '{}',
                '{}', 
                '{}',
                '{}', 
                '{}', 
                '{}'
                );
            """.format(row['id'],row['load_time'],row['rank'], row['points'], row['title'], row['url'], row['comment']))